In [13]:
import tensorflow as tf
import numpy as np


In [2]:
model = tf.keras.applications.ResNet50()
print(model.summary())

102973440/102967424 [==============================] - 10s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_______________________

In [8]:
feats = []
for layer in model.layers:
    if(layer.name[4] == "2"):
        layer1 = layer.output
    if(layer.name[4] == "3"):
        layer2 = layer.output
    if(layer.name[4] == "4"):
        layer3 = layer.output
    if(layer.name[4] == "5"):
        layer4 = layer.output

feats = [layer1, layer2, layer3, layer4]

In [9]:
print(feats)

[<KerasTensor: shape=(None, 56, 56, 256) dtype=float32 (created by layer 'conv2_block3_out')>, <KerasTensor: shape=(None, 28, 28, 512) dtype=float32 (created by layer 'conv3_block4_out')>, <KerasTensor: shape=(None, 14, 14, 1024) dtype=float32 (created by layer 'conv4_block6_out')>, <KerasTensor: shape=(None, 7, 7, 2048) dtype=float32 (created by layer 'conv5_block3_out')>]


In [10]:
activation_model = tf.keras.models.Model(inputs=model.input, outputs=feats)
print(activation_model)

In [12]:
activation_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [16]:
x = np.random.normal(size=(1, 224, 224, 3))

In [21]:
%time
y = activation_model(x)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.81 µs


In [19]:
len(y)

4